In [ ]:
# pip install requests
import requests
from bs4 import BeautifulSoup


url = "https://mykbostats.com/players/2"
apikey = "b327144420a12f944bbdd4863c9fc0bffb28a1e3"
params = {
    "url": url,
    "apikey": apikey,
    "js_render": "true",
    "premium_proxy": "true",
}
response = requests.get("https://api.zenrows.com/v1/", params=params)

#print(response.content[:500])

soup = BeautifulSoup(r.content, 'html.parser')

b'<html lang="en"><head>\n    <meta charset="utf-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge">\n<meta content="fRcZGUQXNy4XUwpSFQUBGiEDFAACJjt98BHX5XTLT9AeyHKNT7lkuRQJ" name="csrf-token">\n\n    <title>Ko Young-min KBO League Batting Stats - Doosan Bears | MyKBO Stats</title>\n\n    <meta name="description" content="Profile, batting stats, recent games and videos for Ko Young-min of the Doosan Bears (KBO League)">\n\n    <meta name="apple-itunes-app" content="app-id=1107341048">\n\n    <meta'


In [29]:
header = soup.select('thead th')

#print(header)

h = header[0]

for i in header: print(i.text)


Year
Team
BA
OBP
SLG
OPS
G
PA
AB
R
H
2B
3B
HR
RBI
SB
CS
BB
SO
TB
GDP
HBP
SH
SF
IBB
RISP
PHBA
Date
Opp
AB
R
H
2B
3B
HR
RBI
BB
HBP
BA
OBP
SLG
OPS
SB
CS
GDP


In [8]:
rows = soup.select('tbody tr')

print(rows[1])

row = rows[0]

name = row.select_one('stick left').text.strip()

print(name)

<tr>
<td class="stick left">2021</td>
<td>
<nobr><span class="team-logo nc"></span>
<a href="/teams/9">NC</a></nobr>
</td>
<td>7.83</td>
<td>1.70</td>
<td>3</td>
<td>2</td>
<td>0</td>
<td>1</td>
<td>0</td>
<td>12</td>
<td>3</td>
<td>0 (0)</td>
<td>0</td>
<td>109</td>
<td>435</td>
<td>23</td>
<td>21</td>
<td>20</td>
<td>25</td>
<td>4</td>
<td>1</td>
<td>4</td>
<td>14</td>
<td>14</td>
<td>0</td>
<td>0</td>
<td>3</td>
<td>0</td>
</tr>


AttributeError: 'NoneType' object has no attribute 'text'